<a href="https://colab.research.google.com/github/gaurivn/Density-Based_Place_Clustering_Using_GeoSN_Data/blob/main/Place_Clustering_Using_GeoSN_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import time
import numpy as np
import pandas as pd
import seaborn as sns

In [17]:
!apt-get install -q libgeos-3.6.2
!apt-get install -q libgeos-dev
!pip install -q https://github.com/matplotlib/basemap/archive/master.zip
!pip install mplleaflet
!pip install wget

Reading package lists...
Building dependency tree...
Reading state information...
libgeos-3.6.2 is already the newest version (3.6.2-1build2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
libgeos-dev is already the newest version (3.6.2-1build2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=3d1d1011400b07a6a278bc150f159c3f2fc3645be218d1f05534733d74c96e6d
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [18]:
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap
from matplotlib import cm
import mplleaflet as mpl
%matplotlib inline

from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [25]:
import numpy as np
np.random.seed(1337)

import os
datadir = './data' 
if not os.path.exists(datadir):
  os.mkdir(datadir)

url = 'https://snap.stanford.edu/data/loc-gowalla_totalCheckins.txt.gz'
filename = os.path.join(datadir, 'loc-gowalla_totalCheckins.txt.gz')

if not os.path.isfile(filename):
  import wget
  wget.download(url, out=filename)

from __future__ import unicode_literals
url = 'https://storage.googleapis.com/google-code-archive-downloads/v2/code.google.com/locrec/gowalla-dataset.zip' 

src_filename = os.path.join(datadir, 'gowalla-dataset.zip')
trg_filename = os.path.join(datadir, 'gowalla_venues.csv') # data/gowalla_venues.csv
import sys  
from imp import reload
if sys.version[0] == '2':
  reload(sys)
  sys.setdefaultencoding("utf-8")
#reload(sys)  
#sys.setdefaultencoding('utf8')

In [28]:
if not os.path.isfile(trg_filename):
  import shutil
  import zipfile
  import re
  wget.download(url, out=src_filename)

  #extract
  zf = zipfile.ZipFile(src_filename, 'r')
  %extracted_file = zf.extract('./../Desktop/DCPGS/gowalla-dataset/spots.txt', datadir)
  
  #tmp_filename = shutil.copy(extracted_file, datadir) # extracfile -> ./data/
  tmp_filename = "data/spots.txt"
  shutil.rmtree(os.path.dirname(extracted_file))
  
  #preprocess
  regex = re.compile(r'POINT\((\-?\d+\.\d+)\s+(\-?\d+\.\d+)\)')
  
  with open(tmp_filename, "r") as src, \
  open(trg_filename, "w") as dst  :
    next(src) 
    for line in src:
      dst.write(regex.sub(r'\2\t\1', line))
  
  #cleanup
  os.remove(src_filename)
  os.remove(tmp_filename)

KeyError: ignored